# Feed-forwardネットワークを用いたフレームごとの音素認識

Feed-forwardネットワークは、入力層-隠れ層-出力層の順にデータが流れる、もっとも基本的なニューラルネットです。多層パーセプトロン(MLP; Multy-Layer Perceptron)とも呼ばれます。

ここではfeed-forwardネットワークを用いてフレームごとに音素の識別を行います。入力層のユニット数は特徴ベクトルの次元数(=26)。出力層のユニット数は音素の数(=41)で、出力の値は各音素である確率になるように学習します(後述)。

隠れ層のユニット数`n_mid_units`は、ニューラルネットが表現できる識別境界の複雑さを制御する非常に重要な条件です。小さすぎると近似が粗くなりすぎ、大きすぎると訓練データに特化しすぎてテストデータに対する性能が落ちます。これは後で実験により調整します。


## ネットワーク構造の記述

Feed-forwardネットワークでは、ネットワークの出力は、入力に対する隠れ層の出力を次の隠れ層に入力し、その出力を次の隠れ層に…と繰り返して計算します。

`l1`は入力から1番目の隠れ層への結合を表しており、実体は $F$ 次元ベクトルから $H$ (=`n_mid_units`) 次元ベクトルへの線形変換(正確にいうとアフィン変換)ですから、  $F\times H$ 行列です。

`l2`は1番目の隠れ層から2番目の隠れ層への結合を表す $H\times H$ 行列です。

`l3`は2番目の隠れ層から出力層への結合を表す $H\times D$ 行列です。($D$ は音素の数=41)

隠れ層の出力は、線形変換の後に活性化関数に通すことで計算できます。今回はReLU関数を使います。入力`x`を`l1`で変換してReLUに通して1番目の隠れ層の出力`h1`を計算し、次に`h1`を`l2`で変換してReLUに通して2番目の出力`h2`を計算し、最後に`h2`を`l3`に通して出力を計算します。(最後の出力をさらに変換することもあるのですが、ここでは省略しています)

In [0]:
import chainer
import chainer.links as L
import chainer.functions as F

class MLP(chainer.Chain):

    def __init__(self, n_mid_units=100, n_out=41):
        super(MLP, self).__init__()

        # パラメータを持つ層の登録
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        x1 = F.concat(x, axis=0)
        h1 = F.relu(self.l1(x1))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

`converter`では、上記`MLP`で読み込めるよう、データを行列のリストに変換しています。

In [0]:
from chainer.dataset import to_device
from chainer.dataset import concat_examples
import numpy as np

def converter(batch, device):
    # alternative to chainer.dataset.concat_examples
    
    Xs = [to_device(device, X) for X, _, __ in batch]
    ts = [to_device(device, t) for _, t, __ in batch]

    lab_batch = [lab.astype(np.int32) for _, __, lab in batch]
    labs = concat_examples(lab_batch, device, padding=0)
    
    return Xs, ts, labs

## 出力の計算と損失関数

音素のone-hot表現は、音素の数(=41)と同じ次元を持つ、要素1つだけが1、ほかが0の値を持つベクトルです。例えば音素 /a/ の番号は6なので、(先頭を0番目として)6番目だけが1、それ以外は0です。

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
with open('phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
ohe = OneHotEncoder()
ohe.fit_transform(le.fit_transform(phones).reshape(-1,1))
ohe.transform([le.transform(['a'])]).toarray()

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

`calculate_loss` 関数を定義します。この関数は、入力をネットワークに入れた出力を計算し、出力と正解ラベルから損失を計算します。

バッチから`converter`で変換したデータ`Xs` (特徴ベクトル系列`X`の発話数分のリスト)を上で定義したネットワークに入れると出力`ys`が得られます。`ys`の各行`y`は、そのフレームに対する音素認識の結果を表します。

一方、`ts`は各フレームの正解音素番号からなるベクトルです。
`y`が音素のone-hot表現 $t_\text{onehot}$ と一致するのが理想ですから、`y`が $t_\text{onehot}$ と似ていないほど大きくなるような「損失関数」を定義し、計算される損失が小さくなるようにネットワークのパラメータ(ここでは`l1`, `l2`, `l3`)を調整します。分類問題では一般に交差エントロピーが損失関数として使われます。これを計算するのが `softmax_cross_entropy` 関数です。

ついでに `accuracy` 関数で正解率も計算しておきます。

In [0]:
import numpy as np
from chainer.backends import cuda

def calculate_loss(net, batch, gpu_id=0, train_mode=True):
    numframes = [X.shape[0] for (X, t, lab) in batch]
    
    Xs, ts, _ = converter(batch, gpu_id) # 生ラベルは使わない

    with chainer.using_config('train', train_mode), \
         chainer.using_config('enable_backprop', train_mode):
        ys = net(Xs)
    
    ts = F.concat(ts, axis=0)
    ts = ts.reshape(ts.shape[0],)
    
    # ロスを計算
    loss = F.softmax_cross_entropy(ys, ts)

    # 精度を計算
    accuracy = F.accuracy(ys, ts)

    return loss, accuracy

乱数を初期化する関数を定義します。

In [0]:
import random
import numpy as np
import chainer

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if gpu_id >= 0:
        chainer.cuda.cupy.random.seed(seed)

`phones` というファイルから音素リストを読み込んでおきます。

In [0]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

with open('phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
le.fit(phones)
nsymbol = len(phones)

## ネットワークの学習

以下がネットワークを学習するプログラムです。

設定可能なパラメータを上の方にまとめておきました。

1. `gpu_id`:
0にするとGPUで実行します。GPUがない場合は-1とします。
1. `batchsize`:
確率的勾配降下法(SGD)などを用いて学習する場合には、データをいくつかの固まり(ミニバッチ)に分割し、それぞれのミニバッチで1回のパラメータ更新を実行します。`batchsize`はミニバッチの大きさで、GPUに乗る限り大きくした方が計算が速くなります。
1. `max_epoch`:
学習データ全体にわたってミニバッチ学習を1回終えることを1エポックといいます。`max_epoch`は学習回数で、何エポック学習するかを設定します。
1. `n_mid_units`:
中間層ユニット数です。

乱数の種をセットするのは、毎回同じ結果が出るようにするためです。初期値がまずいと学習がうまく進まないことがあります。おかしいと思った場合は種を変えてみてください。

学習に使う最適化器(optimizer)には確率的勾配降下法を指定し、学習率は0.01にしています。

whileループが学習のループです。ニューラルネットの学習で使われる勾配降下法は繰り返しアルゴリズムですので、何度も学習を繰り返してパラメータを調整します。

ミニバッチに対して損失 `loss` を計算します。

ニューラルネットの学習では、一般に誤差逆伝播アルゴリズムが使われます。`loss` に対して `backward()` メソッドを実行すると、誤差逆伝播アルゴリズムによりパラメータ更新に必要な誤差が計算されます。次に `optimizer.update()` メソッドを実行すると、計算された誤差をもとにパラメータが更新されます。これでミニバッチ1回の学習が終わりです。

In [8]:
import numpy as np
from chainer import iterators
from chainer import optimizers
from chainer.backends import cuda
from chainer.cuda import to_cpu

gpu_id = 0 # CPUを用いる場合は、この値を-1にしてください

batchsize = 100
max_epoch = 400
n_mid_units = 200

xp = np
if gpu_id >= 0:
    xp = chainer.cuda.cupy

reset_seed(0) # 乱数の種をセット

train = np.load("MHT-train.npy") # train: 450 x (framelen x 26 , framelen, maxphonenum)
test = np.load("MHT-test.npy")

train_iter = iterators.SerialIterator(train, batchsize)

net = MLP(n_mid_units=n_mid_units, n_out=nsymbol)

optimizer = optimizers.SGD(lr=0.01).setup(net)

if gpu_id >= 0:
    net.to_gpu(gpu_id)
    
while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()

    loss, _ = calculate_loss(net, train_batch, gpu_id, train_mode = True)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # バッチ単位で古い記憶を削除し、計算コストを削減する。
    loss.unchain_backward()

    # パラメータの更新
    optimizer.update()
    
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら
        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []
        
        valid_loss, valid_accuracy = calculate_loss(net, test, gpu_id, train_mode=False)
        valid_losses.append(to_cpu(valid_loss.array))
        valid_accuracies.append(to_cpu(valid_accuracy.array))

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))

epoch:01 train_loss:2.9459 val_loss:2.8010 val_accuracy:0.3503
epoch:02 train_loss:2.4236 val_loss:2.4030 val_accuracy:0.4097
epoch:03 train_loss:2.1578 val_loss:2.1621 val_accuracy:0.4355
epoch:04 train_loss:2.0519 val_loss:2.0399 val_accuracy:0.4564
epoch:05 train_loss:1.9509 val_loss:1.9366 val_accuracy:0.4724
epoch:06 train_loss:1.8821 val_loss:1.8804 val_accuracy:0.4830
epoch:07 train_loss:1.7901 val_loss:1.8101 val_accuracy:0.4922
epoch:08 train_loss:1.7337 val_loss:1.7659 val_accuracy:0.4995
epoch:09 train_loss:1.6996 val_loss:1.7261 val_accuracy:0.5035
epoch:10 train_loss:1.6756 val_loss:1.6889 val_accuracy:0.5105
epoch:11 train_loss:1.6557 val_loss:1.6530 val_accuracy:0.5153
epoch:12 train_loss:1.5956 val_loss:1.6309 val_accuracy:0.5176
epoch:13 train_loss:1.5997 val_loss:1.6006 val_accuracy:0.5225
epoch:14 train_loss:1.5612 val_loss:1.5884 val_accuracy:0.5245
epoch:15 train_loss:1.5385 val_loss:1.5665 val_accuracy:0.5276
epoch:16 train_loss:1.5525 val_loss:1.5442 val_accuracy

学習が進むと、訓練データに対する損失が下がって行くのがわかります。テストデータに対する損失も同様に下がって行けば学習はうまく行っています。

最終的には、フレームごとの音素認識率は65%程度になりました。

## 課題
1. 中間層ユニット数を変えて、認識精度がどのように変化するかを検討してください。
1. 隠れ層の数を増やして、認識精度がどのように変化するかを検討してください。

## 音素認識結果の観察

テストデータの0番目 (Jセット第1文) の各フレームに対して、学習したネットワークにより音素認識を実行します。

In [0]:
test_utterance_number = 0

if gpu_id >= 0:
    net.to_gpu(gpu_id)

Xs, ts, _ = converter(test, gpu_id)

# テストデータを1つ取り出します
X_test = Xs[test_utterance_number]
t_test = ts[test_utterance_number]

with chainer.using_config('train', False), \
     chainer.using_config('enable_backprop', False):
    y_test = net([X_test])

# Variable形式で出てくるので中身を取り出す
y_test = y_test.array

# 結果をCPUに送る
y_test = to_cpu(y_test)
t_test = to_cpu(t_test)

# 予測確率の最大値のインデックスを見る
pred_label = y_test.argmax(axis=1)

正解ラベルは `t_test` に、認識結果は `pred_label` に入っています。まず正解を確認しておきます。この発話の内容は、「小さな鰻屋に、熱気のようなものがみなぎる」です。

In [10]:
le.inverse_transform(t_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'ch', 'ch', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 's', 's', 's', 's', 's', 's', 's', 's', 's',
       's', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'u', 'u', 'u', 'u', 'u', 'u',
       'u', 'u', 'u', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',

以下は認識結果です。

In [11]:
le.inverse_transform(pred_label)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 't', 'u', 'I',
       'pau', 'pau', 'pau', 'pau', 'pau', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'sil', 'i', 'ky', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 'j', 's', 's', 's', 's', 's', 's', 's', 's',
       's', 'a', 'a', 'a', 'a', 'a', 'N', 'N', 'n', 'n', 'n', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'o', 'u', 'u', 'u', 'u', 'u', 'o',
       'o', 'N', 'N', 'N', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a', 'a',
       'a', 'e', 'e', 'e', 'N', 'g', 'n', 'g', 'g', 'g', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'e', 'e', 'e', 'e', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'r', 'n', 'n', 'n', 'n', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'sil', 'pau', 'pau', 'sil', 'pau', 'pau', 'pau', 'pau', 

まあまあ認識できているようです。

しかし、少し変なところもあります。例えば sil という音素は必ず発話の先頭と末尾に現れ、そこ以外には来ないはずですが、途中に sil が現れたり、発話の先頭と末尾に pau (発話内ポーズ) が現れています。また、本来は1つの音素は数フレームにわたって連続して現れるのが普通ですが、この認識結果ではところどころ音素が激しく入れ替わっています。これは、ここで行っているフレーム単位の認識は、前後関係を一切考慮していないことが原因です。